In [14]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os, shutil

from config import args
from models.video_net import VideoNet
from data.lrs3_dataset import LRS3Main
from data.utils import collate_fn
from utils.general import num_params, train, evaluate
from tqdm import tqdm
from sys import exit

In [15]:
matplotlib.use("Agg")
np.random.seed(args["SEED"])
torch.manual_seed(args["SEED"])
gpuAvailable = torch.cuda.is_available()
device = torch.device("cuda" if gpuAvailable else "cpu")
kwargs = {"num_workers": args["NUM_WORKERS"], "pin_memory": True} if gpuAvailable else {}
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [16]:
#declaring the train and validation datasets and their corresponding dataloaders
videoParams = {"videoFPS":args["VIDEO_FPS"]}
videoParams

{'videoFPS': 25}

In [16]:
# class LRS2Main(Dataset):

#     """
#     A custom dataset class for the LRS2 main (includes train, val, test) dataset
#     """

#     def __init__(self, dataset, datadir, , charToIx, stepSize, videoParams):
#         super(LRS2Main, self).__init__()
#         with open(datadir + "/" + dataset + ".txt", "r") as f:
#             lines = f.readlines()
#         self.datalist = [datadir + "/main/" + line.strip().split(" ")[0] for line in lines]
#         self.reqInpLen = reqInpLen
#         self.charToIx = charToIx
#         self.dataset = dataset
#         self.stepSize = stepSize
#         self.videoParams = videoParams
#         return


#     def __getitem__(self, index):
#         #using the same procedure as in pretrain dataset class only for the train dataset
#         if self.dataset == "train":
#             base = self.stepSize * np.arange(int(len(self.datalist)/self.stepSize)+1)
#             ixs = base + index
#             ixs = ixs[ixs < len(self.datalist)]
#             index = np.random.choice(ixs)

#         #passing the visual features file and the target file paths to the prepare function to obtain the input tensors
#         visualFeaturesFile = self.datalist[index] + ".npy"
#         targetFile = self.datalist[index] + ".txt"
#         inp, trgt, inpLen, trgtLen = prepare_main_input(visualFeaturesFile, targetFile, self.reqInpLen, self.charToIx, self.videoParams)
#         return inp, trgt, inpLen, trgtLen


#     def __len__(self):
#         #using step size only for train dataset and not for val and test datasets because
#         #the size of val and test datasets is smaller than step size and we generally want to validate and test
#         #on the complete dataset
#         if self.dataset == "train":
#             return self.stepSize
#         else:
#             return len(self.datalist)


In [17]:
print(args["TRAIN_DIRECTORY"])

../lrs3/train_mini/


In [19]:
print(os.path.isdir(args["TRAIN_DIRECTORY"]))

True


In [20]:
dataset = "train"
datadir = args["DATA_DIRECTORY"]
reqInpLen = args["MAIN_REQ_INPUT_LENGTH"]
charToIx = args["CHAR_TO_INDEX"]
stepSize = args["STEP_SIZE"]

In [21]:
trainData = LRS3Main(dataset,datadir,reqInpLen,charToIx,stepSize,videoParams)
trainData.datalist

['../lrs3/train_mini/00j9bKdiOjk/50001',
 '../lrs3/train_mini/00j9bKdiOjk/50002',
 '../lrs3/train_mini/00j9bKdiOjk/50003',
 '../lrs3/train_mini/0af00UcTOSc/50001',
 '../lrs3/train_mini/0af00UcTOSc/50002',
 '../lrs3/train_mini/0af00UcTOSc/50003',
 '../lrs3/train_mini/0af00UcTOSc/50004',
 '../lrs3/train_mini/0af00UcTOSc/50005',
 '../lrs3/train_mini/0af00UcTOSc/50007',
 '../lrs3/train_mini/0af00UcTOSc/50008',
 '../lrs3/train_mini/0af00UcTOSc/50009',
 '../lrs3/train_mini/0af00UcTOSc/50010',
 '../lrs3/train_mini/0af00UcTOSc/50011',
 '../lrs3/train_mini/0af00UcTOSc/50012',
 '../lrs3/train_mini/0af00UcTOSc/50013',
 '../lrs3/train_mini/0af00UcTOSc/50014',
 '../lrs3/train_mini/0akiEFwtkyA/50001',
 '../lrs3/train_mini/0akiEFwtkyA/50002',
 '../lrs3/train_mini/0Amg53UuRqE/50001',
 '../lrs3/train_mini/0Bhk65bYSI0/50001',
 '../lrs3/train_mini/0Bhk65bYSI0/50002',
 '../lrs3/train_mini/0Bhk65bYSI0/50003',
 '../lrs3/train_mini/0Bhk65bYSI0/50004',
 '../lrs3/train_mini/0Bhk65bYSI0/50005',
 '../lrs3/train_

In [8]:
len(trainData.datalist)

163

In [7]:
trainData.__getitem__(trainData.__len__()-1)

(tensor([[1.3807e-02, 0.0000e+00, 0.0000e+00,  ..., 8.9739e-02, 0.0000e+00,
          8.5761e-01],
         [1.4432e-01, 0.0000e+00, 6.5127e-02,  ..., 3.6140e-01, 4.5105e-01,
          8.0595e-02],
         [8.4907e-02, 0.0000e+00, 1.5280e-01,  ..., 4.2185e-02, 3.0149e-01,
          6.3072e-02],
         ...,
         [2.5357e-01, 1.8064e-02, 0.0000e+00,  ..., 2.1392e-01, 3.5996e-04,
          1.8381e-01],
         [9.4345e-02, 0.0000e+00, 0.0000e+00,  ..., 4.2421e-01, 0.0000e+00,
          1.0490e-01],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00]]),
 tensor([13,  8,  2, 10,  8,  1, 17,  5,  7,  1, 10,  2,  5, 17,  9,  1,  8, 13,
          6,  3,  5, 20, 11,  2,  1, 20, 11,  4,  4, 12,  1, 12,  4,  7,  4, 10,
          8,  1, 15,  6,  3,  9,  6,  7,  1,  5,  1, 17,  2, 10,  3,  5,  6,  7,
          1, 10,  5, 12,  6, 13,  8,  1, 12,  6, 10,  2, 17,  3, 11, 14,  1,  5,
          7, 12,  1,  8,  2,  7, 12,  1,  3,  9,  2, 18, 39], dtyp

In [23]:
dataset = "train"
datadir = args["DATA_DIRECTORY"]
reqInpLen = args["MAIN_REQ_INPUT_LENGTH"]
charToIx = args["CHAR_TO_INDEX"]
stepSize = args["STEP_SIZE"]
trainData = LRS3Main(dataset,datadir,reqInpLen,charToIx,stepSize,videoParams)
trainData.datalist
trainLoader = DataLoader(trainData, batch_size=args["BATCH_SIZE"], collate_fn=collate_fn, shuffle=True, **kwargs)
#declaring the model, optimizer, scheduler and the loss function
valData = LRS3Main("val", args["DATA_DIRECTORY"], args["MAIN_REQ_INPUT_LENGTH"], args["CHAR_TO_INDEX"], args["STEP_SIZE"],
                   videoParams)
valLoader = DataLoader(valData, batch_size=args["BATCH_SIZE"], collate_fn=collate_fn, shuffle=True, **kwargs)
model = VideoNet(args["TX_NUM_FEATURES"], args["TX_ATTENTION_HEADS"], args["TX_NUM_LAYERS"], args["PE_MAX_LENGTH"],
                 args["TX_FEEDFORWARD_DIM"], args["TX_DROPOUT"], args["NUM_CLASSES"])
model.to(device)

VideoNet(
  (positionalEncoding): PositionalEncoding()
  (videoEncoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, 

In [46]:
trainLoader.dataset.datalist

['../lrs3/train_mini/00j9bKdiOjk/50001',
 '../lrs3/train_mini/00j9bKdiOjk/50002',
 '../lrs3/train_mini/00j9bKdiOjk/50003',
 '../lrs3/train_mini/0af00UcTOSc/50001',
 '../lrs3/train_mini/0af00UcTOSc/50002',
 '../lrs3/train_mini/0af00UcTOSc/50003',
 '../lrs3/train_mini/0af00UcTOSc/50004',
 '../lrs3/train_mini/0af00UcTOSc/50005',
 '../lrs3/train_mini/0af00UcTOSc/50007',
 '../lrs3/train_mini/0af00UcTOSc/50008',
 '../lrs3/train_mini/0af00UcTOSc/50009',
 '../lrs3/train_mini/0af00UcTOSc/50010',
 '../lrs3/train_mini/0af00UcTOSc/50011',
 '../lrs3/train_mini/0af00UcTOSc/50012',
 '../lrs3/train_mini/0af00UcTOSc/50013',
 '../lrs3/train_mini/0af00UcTOSc/50014',
 '../lrs3/train_mini/0akiEFwtkyA/50001',
 '../lrs3/train_mini/0akiEFwtkyA/50002',
 '../lrs3/train_mini/0Amg53UuRqE/50001',
 '../lrs3/train_mini/0Bhk65bYSI0/50001',
 '../lrs3/train_mini/0Bhk65bYSI0/50002',
 '../lrs3/train_mini/0Bhk65bYSI0/50003',
 '../lrs3/train_mini/0Bhk65bYSI0/50004',
 '../lrs3/train_mini/0Bhk65bYSI0/50005',
 '../lrs3/train_

In [42]:
trainLoader.sampler.data_source.videoParams

{'videoFPS': 25}

In [45]:
for batch, (inputBatch, targetBatch, inputLenBatch, targetLenBatch) in enumerate(tqdm(trainLoader, leave=False, desc="Train",
                                                                                      ncols=75)):
    print(batch)
    print((inputBatch, targetBatch, inputLenBatch, targetLenBatch))

Train:  41%|████████████▍                 | 17/41 [00:00<00:00, 169.97it/s]

0
(tensor([[[0.1171, 0.0226, 0.0000,  ..., 0.0089, 0.0000, 1.1942],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.1662, 0.0000, 0.7076,  ..., 0.2776, 0.0537, 0.0568],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0509, 0.0059, 0.0843,  ..., 0.2364, 0.0211, 0.4227],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        ...,

        [[0.0910, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0424],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
  

34
(tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.1165, 0.0216, 0.0000,  ..., 0.0330, 0.0000, 0.8949],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.1594, 0.3785, 0.0873,  ..., 0.2843, 0.1231, 0.0047],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0870, 0.0000, 0.0000,  ..., 0.2539, 0.0145, 0.3336],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        ...,

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.2260, 0.0147, 0.0108,  ..., 0.1391, 0.3700, 0.0404],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
 

In [29]:
batch = enumerate(trainLoader)
for batch, (inputBatch, targetBatch, inputLenBatch, targetLenBatch) in enumerate(trainLoader):
    print(batch)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40


In [6]:
trainLoader = DataLoader(trainData, batch_size=args["BATCH_SIZE"], collate_fn=collate_fn, shuffle=True, **kwargs)

In [7]:
trainLoader.dataset.datalist

['../lrs3/train_mini/00j9bKdiOjk/50001',
 '../lrs3/train_mini/00j9bKdiOjk/50002',
 '../lrs3/train_mini/00j9bKdiOjk/50003',
 '../lrs3/train_mini/0af00UcTOSc/50001',
 '../lrs3/train_mini/0af00UcTOSc/50002',
 '../lrs3/train_mini/0af00UcTOSc/50003',
 '../lrs3/train_mini/0af00UcTOSc/50004',
 '../lrs3/train_mini/0af00UcTOSc/50005',
 '../lrs3/train_mini/0af00UcTOSc/50007',
 '../lrs3/train_mini/0af00UcTOSc/50008',
 '../lrs3/train_mini/0af00UcTOSc/50009',
 '../lrs3/train_mini/0af00UcTOSc/50010',
 '../lrs3/train_mini/0af00UcTOSc/50011',
 '../lrs3/train_mini/0af00UcTOSc/50012',
 '../lrs3/train_mini/0af00UcTOSc/50013',
 '../lrs3/train_mini/0af00UcTOSc/50014',
 '../lrs3/train_mini/0akiEFwtkyA/50001',
 '../lrs3/train_mini/0akiEFwtkyA/50002',
 '../lrs3/train_mini/0Amg53UuRqE/50001',
 '../lrs3/train_mini/0Bhk65bYSI0/50001',
 '../lrs3/train_mini/0Bhk65bYSI0/50002',
 '../lrs3/train_mini/0Bhk65bYSI0/50003',
 '../lrs3/train_mini/0Bhk65bYSI0/50004',
 '../lrs3/train_mini/0Bhk65bYSI0/50005',
 '../lrs3/train_

In [10]:
valData = LRS3Main("val", args["DATA_DIRECTORY"], args["MAIN_REQ_INPUT_LENGTH"], args["CHAR_TO_INDEX"], args["STEP_SIZE"],
                   videoParams)

In [11]:
valLoader = DataLoader(valData, batch_size=args["BATCH_SIZE"], collate_fn=collate_fn, shuffle=True, **kwargs)

In [12]:
#declaring the model, optimizer, scheduler and the loss function
model = VideoNet(args["TX_NUM_FEATURES"], args["TX_ATTENTION_HEADS"], args["TX_NUM_LAYERS"], args["PE_MAX_LENGTH"],
                 args["TX_FEEDFORWARD_DIM"], args["TX_DROPOUT"], args["NUM_CLASSES"])
model.to(device)

VideoNet(
  (positionalEncoding): PositionalEncoding()
  (videoEncoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, 

In [6]:
optimizer = optim.Adam(model.parameters(), lr=args["INIT_LR"], betas=(args["MOMENTUM1"], args["MOMENTUM2"]))

In [7]:
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=args["LR_SCHEDULER_FACTOR"],
                                                 patience=args["LR_SCHEDULER_WAIT"], threshold=args["LR_SCHEDULER_THRESH"],
                                                 threshold_mode="abs", min_lr=args["FINAL_LR"], verbose=True)

In [8]:
loss_function = nn.CTCLoss(blank=0, zero_infinity=False)

In [16]:
#removing the checkpoints directory if it exists and remaking it
if os.path.exists(args["CODE_DIRECTORY"] + "/checkpoints"):
    while True:
        ch = input("Continue and remove the 'checkpoints' directory? y/n: ")
        if ch == "y":
            break
        elif ch == "n":
            exit()
        else:
            print("Invalid input")
    shutil.rmtree(args["CODE_DIRECTORY"] + "/checkpoints")

Continue and remove the 'checkpoints' directory? y/n: n
Continue and remove the 'checkpoints' directory? y/n: n
Continue and remove the 'checkpoints' directory? y/n: n
Continue and remove the 'checkpoints' directory? y/n: y


In [16]:
os.mkdir(args["CODE_DIRECTORY"] + "/checkpoints")
os.mkdir(args["CODE_DIRECTORY"] + "/checkpoints/models")
os.mkdir(args["CODE_DIRECTORY"] + "/checkpoints/plots")

FileExistsError: [WinError 183] Cannot create a file when that file already exists: '../../avsr_lr3//checkpoints'

In [16]:
#loading the pretrained weights
##TODO can do this
# if args["PRETRAINED_MODEL_FILE"] is not None:
#     print("\n\nPre-trained Model File: %s" %(args["PRETRAINED_MODEL_FILE"]))
#     print("\nLoading the pre-trained model .... \n")
#     model.load_state_dict(torch.load(args["CODE_DIRECTORY"] + args["PRETRAINED_MODEL_FILE"], map_location=device))
#     model.to(device)
#     print("Loading Done.\n")

In [9]:
trainingLossCurve = list()
validationLossCurve = list()
trainingWERCurve = list()
validationWERCurve = list()

In [10]:
#printing the total and trainable parameters in the model
numTotalParams, numTrainableParams = num_params(model)
print("\nNumber of total parameters in the model = %d" %(numTotalParams))
print("Number of trainable parameters in the model = %d\n" %(numTrainableParams))


Number of total parameters in the model = 37849128
Number of trainable parameters in the model = 37849128



In [12]:
args["NUM_STEPS"] = 5

In [13]:
print("\nTraining the model .... \n")

trainParams = {"spaceIx":args["CHAR_TO_INDEX"][" "], "eosIx":args["CHAR_TO_INDEX"]["<EOS>"]}
valParams = {"decodeScheme":"greedy", "spaceIx":args["CHAR_TO_INDEX"][" "], "eosIx":args["CHAR_TO_INDEX"]["<EOS>"]}

for step in range(args["NUM_STEPS"]):
#     print("Total step: " str(args["NUM_STEPS"]) + " Current Step: " + str(step))
    #train the model for one step
    trainingLoss, trainingCER, trainingWER = train(model, trainLoader, optimizer, loss_function, device, trainParams)
    trainingLossCurve.append(trainingLoss)
    trainingWERCurve.append(trainingWER)

    #evaluate the model on validation set
    validationLoss, validationCER, validationWER = evaluate(model, valLoader, loss_function, device, valParams)
    validationLossCurve.append(validationLoss)
    validationWERCurve.append(validationWER)

    #printing the stats after each step
    print("Step: %03d || Tr.Loss: %.6f  Val.Loss: %.6f || Tr.CER: %.3f  Val.CER: %.3f || Tr.WER: %.3f  Val.WER: %.3f"
          %(step, trainingLoss, validationLoss, trainingCER, validationCER, trainingWER, validationWER))

    #make a scheduler step
    scheduler.step(validationWER)


    #saving the model weights and loss/metric curves in the checkpoints directory after every few steps
    if ((step%args["SAVE_FREQUENCY"] == 0) or (step == args["NUM_STEPS"]-1)) and (step != 0):

        savePath = args["CODE_DIRECTORY"] + "/checkpoints/models/train-step_{:04d}-wer_{:.3f}.pt".format(step, validationWER)
        torch.save(model.state_dict(), savePath)

        plt.figure()
        plt.title("Loss Curves")
        plt.xlabel("Step No.")
        plt.ylabel("Loss value")
        plt.plot(list(range(1, len(trainingLossCurve)+1)), trainingLossCurve, "blue", label="Train")
        plt.plot(list(range(1, len(validationLossCurve)+1)), validationLossCurve, "red", label="Validation")
        plt.legend()
        plt.savefig(args["CODE_DIRECTORY"] + "/checkpoints/plots/train-step_{:04d}-loss.png".format(step))
        plt.close()

        plt.figure()
        plt.title("WER Curves")
        plt.xlabel("Step No.")
        plt.ylabel("WER")
        plt.plot(list(range(1, len(trainingWERCurve)+1)), trainingWERCurve, "blue", label="Train")
        plt.plot(list(range(1, len(validationWERCurve)+1)), validationWERCurve, "red", label="Validation")
        plt.legend()
        plt.savefig(args["CODE_DIRECTORY"] + "/checkpoints/plots/train-step_{:04d}-wer.png".format(step))
        plt.close()


print("\nTraining Done.\n")


Training the model .... 



Step: 000 || Tr.Loss: 3.716386  Val.Loss: 3.301039 || Tr.CER: 1.000  Val.CER: 1.000 || Tr.WER: 1.000  Val.WER: 1.000


Step: 001 || Tr.Loss: 3.359583  Val.Loss: 3.296175 || Tr.CER: 1.000  Val.CER: 1.000 || Tr.WER: 1.000  Val.WER: 1.000


Step: 002 || Tr.Loss: 3.308600  Val.Loss: 3.448914 || Tr.CER: 1.000  Val.CER: 1.000 || Tr.WER: 1.000  Val.WER: 1.000


Step: 003 || Tr.Loss: 3.314058  Val.Loss: 3.357716 || Tr.CER: 1.000  Val.CER: 1.000 || Tr.WER: 1.000  Val.WER: 1.000


Step: 004 || Tr.Loss: 3.299858  Val.Loss: 3.314137 || Tr.CER: 1.000  Val.CER: 1.000 || Tr.WER: 1.000  Val.WER: 1.000

Training Done.

